In [4]:
import requests
import pandas as pd
import time
import random
from bs4 import BeautifulSoup

# ---------- CONFIG ----------
BASE_SEARCH_URL = "https://openlibrary.org/search.json"
BASE_BOOK_URL = "https://openlibrary.org{book_path}"
GENRES = [
    "fantasy", "romance", "fiction", "non-fiction",
    "history", "biography", "science", "humour", "memoir"
]
OUTPUT_FILE = "openlibrary_books_with_meta_ratings2.csv"
# ----------------------------

def get_openlibrary_books(genre, limit=100):
    """Fetch books for a given genre."""
    params = {"q": genre, "limit": limit}
    try:
        r = requests.get(BASE_SEARCH_URL, params=params, timeout=10)
        r.raise_for_status()
        return r.json().get("docs", [])
    except Exception as e:
        print(f"⚠️ Failed to fetch '{genre}': {e}")
        return []

def get_rating_from_page(book_path):
    """Extract rating and rating count from <meta> tags."""
    try:
        url = BASE_BOOK_URL.format(book_path=book_path)
        r = requests.get(url, timeout=10)
        if r.status_code != 200:
            return None, None
        soup = BeautifulSoup(r.text, "html.parser")

        rating_meta = soup.find("meta", {"itemprop": "ratingValue"})
        count_meta = soup.find("meta", {"itemprop": "ratingCount"})

        rating = float(rating_meta["content"]) if rating_meta and rating_meta.get("content") else None
        count = int(count_meta["content"]) if count_meta and count_meta.get("content") else None

        return rating, count
    except Exception:
        return None, None


books = []

for genre in GENRES:
    print(f"\n📚 Fetching genre: {genre}")
    docs = get_openlibrary_books(genre, limit=120)
    print(f"   → Found {len(docs)} items")

    for d in docs:
        title = d.get("title")
        authors = ", ".join(d.get("author_name", [])) if d.get("author_name") else None
        pub_year = d.get("first_publish_year")

        # prefer edition key, fallback to work key
        if d.get("edition_key"):
            book_path = f"/books/{d['edition_key'][0]}"
        elif d.get("key"):
            book_path = d["key"]
        else:
            continue

        rating, count = get_rating_from_page(book_path)
        time.sleep(random.uniform(1.0, 1.5))  # polite delay

        books.append({
            "Genre": genre,
            "Title": title,
            "Author": authors,
            "Average Rating": rating,
            "Ratings Count": count,
            "Publication Year": pub_year,
            "Link": f"https://openlibrary.org{book_path}"
        })

    print(f"   ✅ Total collected so far: {len(books)}")
    time.sleep(random.uniform(2.0, 3.0))  # delay between genres

# Save to CSV
df = pd.DataFrame(books)
df.drop_duplicates(subset=["Title", "Author"], inplace=True)
df.to_csv(OUTPUT_FILE, index=False)

print(f"\n🎉 Done! Saved {len(df)} unique books → {OUTPUT_FILE}")



📚 Fetching genre: fantasy
   → Found 120 items
   ✅ Total collected so far: 120

📚 Fetching genre: romance
   → Found 120 items
   ✅ Total collected so far: 240

📚 Fetching genre: fiction
   → Found 120 items
   ✅ Total collected so far: 360

📚 Fetching genre: non-fiction
   → Found 120 items
   ✅ Total collected so far: 480

📚 Fetching genre: history
   → Found 120 items
   ✅ Total collected so far: 600

📚 Fetching genre: biography
   → Found 120 items
   ✅ Total collected so far: 720

📚 Fetching genre: science
   → Found 120 items
   ✅ Total collected so far: 840

📚 Fetching genre: humour
   → Found 120 items
   ✅ Total collected so far: 960

📚 Fetching genre: memoir
   → Found 120 items
   ✅ Total collected so far: 1080

🎉 Done! Saved 657 unique books → openlibrary_books_with_meta_ratings2.csv
